# Implement shooting method to solve for time-path of variables

In [214]:
import numpy as np
from scipy.optimize import fsolve

In [215]:
# Define parameters
rho = 0.99
r = 0.05
a = 1
delta = 0.01
alpha = 0.7
gamma = 0.7
sigma = 0.7
params = (rho, r, a, delta, alpha, gamma, sigma)

# Initial conditions
K_0 = 2
A_0 = 3
mu_0 = 0.2
p_0 = 0.3
state_0 = (K_0, A_0, mu_0, p_0)

In [216]:
# Define g(x) and g'(x) (labor market equilibrium tradeoff) function
def g(x):
    k = 5/4
    s = 1/2
    a = 1/(k**0.5 - s)
    y = k - (x/a + s)**2
    
    return y

def g_prime(x):
    k = 5/4
    s = 1/2
    a = 1/(k**0.5 - s)
    y = -2*(x/a + s)/a
    
    return y

In [217]:
# Define utility functions
def u(c, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    
    return (c**(1-alpha))/(1-alpha)

def u_prime(c, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    
    return c**(-alpha)

def v(l, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    
    return (l**(1-gamma))/(1-gamma)

def v_prime(l, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    
    return l**(-gamma)

def B(A, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    
    return (A**(1-sigma))/(1-sigma)

def B_prime(A, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    
    if A <= 0:
        return 1e999
    
    return A**(-sigma)

In [218]:
# Define functions to compute FOCs
# Consumption FOC
def c_foc(state, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    K, A, mu, p = state # Unpack state vector
    
    return mu**(-1/alpha)

# Leisure FOC
def l_rhs(x, state, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    K, A, mu, p = state # Unpack state vector
    
    rhs = mu*K*g(x) + p*a*K*x
    
    return rhs

# Job choice FOC
def x_lhs(l, x, state, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    K, A, mu, p = state # Unpack state vector
    
    h = 1-l
    lhs = h*(a*p*K + mu*K*g_prime(x))
    
    return lhs

In [219]:
# Define function that checks FOCs
def l_foc(l, x, state, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    K, A, mu, p = state # Unpack state vector
    
    lhs = v_prime(l, params)
    rhs = l_rhs(x, state, params)
    diff = lhs - rhs
    
    return diff

def x_foc(l, x, state, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    K, A, mu, p = state # Unpack state vector
    
    diff = x_lhs(l, x, state, params) - 0
    
    return diff

In [220]:
# Combine both FOC checkers within one function
def lx_foc(guess, state, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    K, A, mu, p = state # Unpack state vector
    l, x = guess
    
    res1 = l_foc(l, x, state, params) # Check FOC for l
    res2 = x_foc(l, x, state, params) # Check FOC for x
    
    error = np.array([res1, res2])
    
    return error

In [221]:
# Define a function that solves for l and x for a given mu and p from FOCs
def foc_solve(state, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    K, A, mu, p = state # Unpack state vector
    
    # Find c first
    c = c_foc(state, params)
    
    # Try looking for interior solutions first
    [l, x] = fsolve(lx_foc, [0.5, 0.5], args = (state, params))
    
    # Check if solutions are interior
    if 0 < l < 1 and 0 < x < 1:
        print("Interior solution obtained")
        return np.array([l, x, c])
    
    # If not interior, try boundary solutions for x
    # Try x = 0
    x = 0
    [l] = fsolve(l_foc, [0.5], args = (x, state, params))
    # Check if l is valid
    if 0 < l < 1:
        # Check FOC for x holds
        check = x_lhs(l, x, state, params)
        if check <= 0:
            print("x = 0 is a valid boundary solution")
            return np.array([l, x, c])
        
    # If l is not valid, try x = 1
    x = 1
    [l] = fsolve(l_foc, [0.5], args = (x, state, params))
    if 0 < l < 1:
        # Check FOC for x holds
        check = x_lhs(l, x, state, params)
        if check >= 0:
            print("x = 1 is a valid boundary solution")
            return np.array([l, x, c])
    
    # If neither x = 0 nor x = 1 are valid, check the solution l = 1 (x can be anything)
    l = 1
    x = 1
    check = l_foc(l, x, state, params)
    if check >= 0:
        print("l = 1 is a valid boundary solution")
        return np.array([l, x, c])
    else:
        print("No valid solution???")

In [222]:
# Define functions that produce state tomorrow given today's solution
def state_iterate(choice, state, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    K, A, mu, p = state # Unpack state vector
    l, x, c = choice
    
    h = 1 - l
    mu_1 = (1 + rho - r)*mu
    p_1 = p*(1 + rho + delta - a*x*h) - g(x)*h*mu
    A_1 = (1 + r)*A + g(x)*h*K - c
    K_1 = (1 + a*x*h - delta)*K
    
    return np.array([K_1, A_1, mu_1, p_1])

In [223]:
def iterate(state_0, T, params):
    # Define arrays to store results
    choice_path = np.zeros((3, T))
    state_path = np.zeros((4, T+1))

    state_path[:,0] = state_0

    # Try iterating forward in time given initial conditions
    for i in range(T):
        state = state_path[:, i]
        choice = foc_solve(state, params)
        choice_path[:,i] = choice

        state_path[:, i+1] = state_iterate(choice, state, params)
        
    return (choice_path, state_path)

In [224]:
def terminal_condition(guess, K_0, A_0, T, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    mu, p = guess # Unpack guess
    state_0 = [K_0, A_0, mu, p]
    choice_path, state_path = iterate(state_0, T, params)
    
    K, A, mu, p = state_path[:, -1]
    
    err1 = K*p
    err2 = mu - B_prime(A, params)
    
    return np.array([err1, err2])

In [225]:
[mu_0, p_0] = fsolve(terminal_condition, [0.5, 0.5], args=(K_0, A_0, T, params))

Interior solution obtained
Interior solution obtained
Interior solution obtained
x = 0 is a valid boundary solution
x = 0 is a valid boundary solution
Interior solution obtained
Interior solution obtained
Interior solution obtained
x = 0 is a valid boundary solution
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
x = 0 is a valid boundary solution
x = 0 is a valid boundary solution
Interior solution obtained
Interior solution obtained
Interior solution obtained
x = 0 is a valid boundary solution
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interio

C:\Users\churn\Utilities\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\churn\Utilities\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in power


In [226]:
state_sol = [K_0, A_0, mu_0, p_0]

In [228]:
[choice_path, state_path] = iterate(state_0, T, params)

l = 1 is a valid boundary solution
x = 1 is a valid boundary solution
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
x = 0 is a valid boundary solution
x = 0 is a valid boundary solution
x = 0 is a valid boundary solution
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained
Interior solution obtained


C:\Users\churn\Utilities\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\churn\Utilities\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in power


In [230]:
choice_path[0,:]

array([1.        , 0.78184267, 0.5       , 0.5       , 0.5       ,
       0.5       , 0.5       , 0.5       , 0.5       , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ])